#### The Crawler Inverted Index Construction and Relevant Link Retrieval 

## Instuction
Please exucate all the cells in the notebook and navigate your search with these 3 functions: return_top_15_links / term_freq_finder / inverted_index_info 

In [1]:
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import PorterStemmer
from math import log
import numpy as np

#### Import the 5000 web pages to the notebook

In [2]:
with open('[CSC575 Final Project Rae Chiang pickle file] 5000_links_texts.pickle', 'rb') as handle:
    dict_= pickle.load(handle)

In [3]:
len(dict_)

5000

#### Define functions to clean the text (tokenization /  bigram/ lowercase /remove punctuation /stemming)

In [4]:
stemmer=PorterStemmer()
def process_term(term_lst):
    collect=[]
    for i in term_lst:
        l=i.split()
        term_lst=i.split()
        for term in term_lst:
            collect.append(term)
                
    #some punctuation cleaning after the tokenization 
    clean_collect=[]
    for t in collect:
        if ":" in t:
            t=t.rstrip(':')
        if "," in t:
            t=t.rstrip(',')
        if "." in t:
            t=t.rstrip('.')
        if "(" in t:           #terms like "(d2l)"
            t=t.lstrip('(')
            try:
                t=t.rstrip(')')
            except:
                continue    
        #case folding
        term=t.lower()
        
        #stemming 
        if term not in stopwords.words("english"):
            term=stemmer.stem(term)
            if term.isalpha():
                term=clean_collect.append(term)
            else:
                if term.split("’")[0] != term: #filter out some puntuation
                    term=term.split("’")[0]    #terms like "you’re"
                    clean_collect.append(term)

    
    return clean_collect


#formating
def process_dict(dictionary):
    tokenize_dict={}
    for link in dictionary.keys():
        tokenize_dict[link]=process_term(dictionary[link])
        
    return tokenize_dict

#### Inverted Index Construction

In [5]:
#term freq for the corpus
clean=process_dict(dict_)
clean_term=[term for link in clean.values() for term in link]
term_freq=nltk.FreqDist(clean_term)

#assign link_id for each link
doc_id={}
for index,link in enumerate(clean.keys(),1):
    doc_id[index]=link

    
#posting list construction
term_freq_inDoc={} #term freq for each link doc
for index,link in doc_id.items():
    term_freq_inDoc[index]=nltk.FreqDist(clean[link])

posting={}
for term in term_freq.keys():
    doc_freq_collect_P={}
    for index in term_freq_inDoc.keys():
        if term in term_freq_inDoc[index].keys():
            doc_freq_collect_P[index]=term_freq_inDoc[index][term]

    posting[term]=doc_freq_collect_P

#inverted index construction [total freq/N Doc/ posting lst (doc#/freq)]
inverted_index={}
for term in term_freq.keys():                    
    inverted_index[term]=(term_freq[term],len(posting[term]),posting[term]) #[N Doc / total freq / posting lst (doc#/freq)]
    
#calculating idf
# doc freq for each term 
idf={}
for term in term_freq.keys():
    idf[term]=log(len(doc_id)/len(posting[term]))


#construct the posting list with tfidf score with ltc schema
# the tf here is 1+log(tf)
#the log is based 10

H_tfidf_posting={}
for term in term_freq.keys():
    doc_freq_collect_H={}
    for index in term_freq_inDoc.keys():
        if term in term_freq_inDoc[index].keys():
            doc_freq_collect_H[index]=idf[term]*(1+log(term_freq_inDoc[index][term]))
            
    H_tfidf_posting[term]=doc_freq_collect_H
    
    
#calculate the d lenth for each doc/link
d_length={}
for doc_index in doc_id.keys(): 
    for t in H_tfidf_posting.keys():
        
        # if t appears in the doc or not
        if doc_index in H_tfidf_posting[t].keys():
            
            # if t in the doc then square the weight and accumulate it in the dict  
            d_length[doc_index]=+ H_tfidf_posting[t][doc_index]*H_tfidf_posting[t][doc_index]
            
    # square root the accumulate number to obtain the length
    d_length[doc_index]=np.sqrt(d_length[doc_index])


# normalized the tfidf score
H_tfidf_posting_normalized={}

for term in term_freq.keys():
    collect={}
    for index in H_tfidf_posting[term].keys():
        collect[index]=H_tfidf_posting[term][index]/d_length[index]
    H_tfidf_posting_normalized[term]=collect

#### Enter queries to retrieved the top 10 relevant links based on the cosine similarity score (ltc.ltn schema)

In [19]:
#given the query the function will return the most relevant top 15 links with ltn schema

def return_top_10_links(query):
    term_freq_q=nltk.FreqDist(process_term([query]))
    q_times_d={}
    for term in term_freq_q.keys():
        #make sure term is the corpus inverted index
        if term in idf.keys():

            #tfidf score for query term
            q_tfidf_score=idf[term]*(1+log(term_freq_q[term]))

            for doc,score in H_tfidf_posting_normalized[term].items():
                if bool(q_times_d.keys()):
                    q_times_d[doc]=+score*q_tfidf_score
                else: 
                    q_times_d[doc]=score*q_tfidf_score

    # make sure the score is normalized by the query lenth and doc length
    for docID, numerator in q_times_d.items():
        q_times_d[docID]=numerator
    
    sort=sorted(q_times_d.items(), key=lambda dic: dic[1],reverse=True)[:10]
    link=[]
    for i in sort:
        link.append([i[0],doc_id[i[0]],i[1]])
    print("Top 10 retrieved 1inks:\n (linkID, link,cosine score)")
    
    return link



In [7]:
# Enter the linkID to find the term frequency of the link
def term_freq_finder(linkID):
    return term_freq_inDoc[linkID]

# Enter the term to find the inverted index info of the link
# The retern information is in the format of (N Doc / total freq {posting lst (doc#/freq)})
def inverted_index_info(term):
    return inverted_index[term]

### Make sure to execute all the cells above before move on to the next section

### Exercute the system

In [11]:
# you can assign your query to q, q must be a string
# for example
q="lincoln park campus"

In [ ]:
#excute the function to obtain the top 10 retreval results
return_top_10_links(q)

In [ ]:
# type in the linkID in term_freq_finder function you will be able to find the term frequency in the web page
term_freq_finder(205)

In [ ]:
#type in the term you are interested in to return the inverted index information, the return information is in this format
#[total freq/N Doc/ posting lst (doc#/freq)]
inverted_index_info("campu")